In [23]:
import os
import cv2
import random
import imutils
import argparse 
from glob import  glob
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import math

In [41]:
dirs = glob('train_data/data/*')
min_len = 100000
tot = 0
for dirn in dirs:
    imgs = glob(f'{dirn}/*.png')
    tot += len(imgs)
    min_len = min(min_len, len(imgs))
print(min_len)

360 52939


In [25]:
dataset_path = 'train_data/data'
dir_names = [path.split('/')[2] for path in glob(f'{dataset_path}/*')]
target_img_size = (100, 100)
sample_count = 350
print(dir_names)

['a_1', '16_b_r', 't4', 't2', '1', 'chord', '#', 'dot', 'flat_b', 'dot_b', 't24_b', 'a_4', '2', '8_b_r', '8', 'bar_b', '#_b', 'a_16', '16_b_n', 'a_2', 'natural', '32_b_r', 'flat', 'bar', 'clef_b', 'cross', 'natural_b', '4', '32_b_n', 't24', '8_b_n', '16', 't44', 'a_32', 'clef', 'a_8', 'p', 't44_b']


In [26]:
def extract_raw_pixels(img):
    resized = cv2.resize(img, target_img_size)
    return resized.flatten()

In [27]:
def extract_hsv_histogram(img):
    resized = cv2.resize(img, target_img_size)
    hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
    return hist.flatten()    

In [28]:
def extract_hog_features(img):
    img = cv2.resize(img, target_img_size)
    win_size = (100, 100)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [29]:
def extract_features(img, feature_set='raw'):
    if feature_set == 'hog':
        return extract_hog_features(img)
    elif feature_set == 'raw':
        return extract_raw_pixels(img)
    else:
        return extract_hsv_histogram(img)

In [30]:
def load_dataset(feature_set='raw'):
    features = []
    labels = []
    for dir_name in dir_names:
        print(dir_name)
        imgs = glob(f'{dataset_path}/{dir_name}/*.png')
        subset = random.sample([i for i in range(len(imgs))], sample_count)
        for i in subset:
            img = cv2.imread(imgs[i])
            labels.append(dir_name)
            features.append(extract_features(img, feature_set))
        
    return features, labels  

In [31]:
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'NN': MLPClassifier(solver='sgd', random_state=random_seed, hidden_layer_sizes=(500,), max_iter=20, verbose=1)
}

In [32]:
def run_experiment(classifier='SVM', feature_set='hog'):
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset(feature_set)
    print('Finished loading dataset.')
    
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=random_seed)
    
    model = classifiers[classifier]
    print('############## Training', classifier, "##############")
    model.fit(train_features, train_labels)
    accuracy = model.score(test_features, test_labels)
    print(classifier, 'accuracy:', accuracy*100, '%')

    return model, accuracy

In [33]:
model, accuracy = run_experiment('SVM', 'hog') 
'''
SVM accuracy: 91.52941176470588 %
KNN accuracy: 72.88235294117648 %
NN accuracy: 85.41176470588235 %
'''

Loading dataset. This will take time ...
a_1
16_b_r
t4
t2
1
chord
#
dot
flat_b
dot_b
t24_b
a_4
2
8_b_r
8
bar_b
#_b
a_16
16_b_n
a_2
natural
32_b_r
flat
bar
clef_b
cross
natural_b
4
32_b_n
t24
8_b_n
16
t44
a_32
clef
a_8
p
t44_b
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 94.88721804511279 %


'\nSVM accuracy: 91.52941176470588 %\nKNN accuracy: 72.88235294117648 %\nNN accuracy: 85.41176470588235 %\n'

In [34]:

filename = f'trained_models/svm_trained_model_hog_{accuracy}.sav'
pickle.dump(model, open(filename, 'wb'))

In [40]:
# loaded_model = pickle.load(open(filename, 'rb'))

0


In [36]:
def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [37]:
# test_imgs = glob('testresult/*.png')
# for img in test_imgs:
#     img = cv2.imread(img)
#     features = extract_features(img, 'raw')
#     labels = model.predict([features])
#     show_images([img], labels)

In [38]:
# run_experiment('hsv')
'''
SVM accuracy: 29.117647058823533 %
KNN accuracy: 56.99999999999999 %
NN accuracy: 7.588235294117647 %
'''

'\nSVM accuracy: 29.117647058823533 %\nKNN accuracy: 56.99999999999999 %\nNN accuracy: 7.588235294117647 %\n'

In [39]:
# run_experiment('raw')
'''
SVM accuracy: 80.47058823529412 %
KNN accuracy: 83.70588235294117 %
NN accuracy: 9.294117647058824 %
'''

'\nSVM accuracy: 80.47058823529412 %\nKNN accuracy: 83.70588235294117 %\nNN accuracy: 9.294117647058824 %\n'